In [8]:
import pennylane as qml
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from math import log, ceil
from tqdm import tqdm
import torch.multiprocessing as mp
import numpy as np

In [2]:
dataset = Planetoid("/tmp/Cora", "Cora", split="random", num_train_per_class=int(140/7))
(
    dataset.train_mask.sum().item(),
    dataset.val_mask.sum().item(),
    dataset.test_mask.sum().item()
)

(140, 500, 1000)

In [3]:
n = ceil(log(dataset.num_features)/log(2))
n

11

In [4]:
dev = qml.device("default.qubit", wires=n)

In [5]:
n_out = 7

@qml.qnode(dev, interface="torch")
def circuit(inputs, weights):
    qml.AmplitudeEmbedding(inputs, wires=range(n), pad_with=0., normalize=True)
    qml.StronglyEntanglingLayers(weights=weights, wires=range(n))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_out)]

In [12]:
# def single(layer, x):
#     return layer(x)

class QNNModel(torch.nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        
        self.q_layer_1 = qml.qnn.TorchLayer(circuit, {
            "weights": (n_layers, n, 3)
        })

    def single(self, x):
        return self.q_layer_1(x)
    
    def forward(self, x):
        out = torch.zeros(x.shape[0], n_out)

        for i in tqdm(range(x.shape[0])):
            out[i] = self.q_layer_1(x[i])
        
        # with mp.Pool() as p:
        #     res = [p.apply_async(single, args=(self.q_layer_1, x)) for x in data.x]
        #     for i in tqdm(range(data.x.shape[0])):
        #         out[i] = res[i].get()
            
        return out

In [17]:
device = "cpu" # torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = QNNModel(5).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    print("Epoch", epoch)
    optimizer.zero_grad()
    out = model(data.x[data.train_mask])
    loss = F.nll_loss(out, data.y[data.train_mask])
    loss.backward()
    optimizer.step()

Epoch 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:12<00:00, 10.94it/s]


Epoch 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:13<00:00, 10.63it/s]


Epoch 2


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:13<00:00, 10.74it/s]


Epoch 198


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:13<00:00, 10.63it/s]


Epoch 199


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:13<00:00, 10.56it/s]


In [18]:
model.eval()
pred = model(data.x[data.test_mask]).argmax(dim=1)
correct = (pred == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:29<00:00, 11.14it/s]


Accuracy: 0.1340
